In [1]:
%matplotlib inline
from effective_quadratures.PolyParams import PolynomialParam
from effective_quadratures.PolyParentFile import PolyParent
from effective_quadratures.IndexSets import IndexSet
from effective_quadratures.Utils import column, evalfunction
import effective_quadratures.MatrixRoutines as mat
import effective_quadratures.ComputeStats as stats
from effective_quadratures.EffectiveQuadSubsampling import EffectiveSubsampling
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

ImportError: No module named effective_quadratures.PolyParams

<h1> Vegetation UQ with Effective-Quadratures

Consider a uniform variation in the following parameters:
    
$$\begin{array}{lll} \hline
Variable & Range & Description \\ \hline
\rho_{shoot} & [38.2, 250.4] & Shoot \; density \; (/m^2)\\ 
L & [0.157, 0.313] & Plant \; length \; (m)\\ 
d & [0.002, 0.01] & Diameter \;  (m)\\ 
t & [0.0002, 0.001] & Thickness \; (m)\\  \hline
\end{array}$$

Being by setting up the ranges for the four parameters

In [2]:
quadratic_pts = 3
x1 = PolynomialParam("Uniform", 38.2, 250.4, [], [], [], 3)
x2 = PolynomialParam("Uniform", 0.157, 0.313, [], [], [], 3)
x3 = PolynomialParam("Uniform", 0.002, 0.01, [], [], [], 3)
x4 = PolynomialParam("Uniform", 0.0002, 0.001, [], [], [], 3)
orders = [2,2,2,2]
x1x2x3x4 = [x1, x2, x3, x4]

Set the polynomial basis; this will dictate how many function evaluations we will require. Bear in mind that a quadratic in 4 dimensions requires $3^4=81$ points. To reduce the cost we will opt for a total order basis.

In [3]:
polybasis = IndexSet("total order", orders)
print IndexSet.getIndexSet(polybasis)
maximum_number_of_evals = IndexSet.getCardinality(polybasis)
print maximum_number_of_evals

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  2.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  1.]
 [ 0.  0.  2.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  1.]
 [ 0.  1.  1.  0.]
 [ 0.  2.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  1.  0.]
 [ 1.  1.  0.  0.]
 [ 2.  0.  0.  0.]]
15


In [6]:
effectiveQuads = EffectiveSubsampling(x1x2x3x4, polybasis, 0)
A, esquad_pts, W, indices = EffectiveSubsampling.getAsubsampled(effectiveQuads, maximum_number_of_evals)
print esquad_pts

[[  1.44300000e+02   2.35000000e-01   6.00000000e-03   6.00000000e-04]
 [  6.21152934e+01   1.74581460e-01   2.90161332e-03   6.00000000e-04]
 [  2.26484707e+02   2.35000000e-01   9.09838668e-03   2.90161332e-04]
 [  6.21152934e+01   2.95418540e-01   6.00000000e-03   9.09838668e-04]
 [  2.26484707e+02   1.74581460e-01   6.00000000e-03   9.09838668e-04]
 [  2.26484707e+02   2.35000000e-01   2.90161332e-03   2.90161332e-04]
 [  6.21152934e+01   2.35000000e-01   9.09838668e-03   2.90161332e-04]
 [  2.26484707e+02   2.95418540e-01   6.00000000e-03   9.09838668e-04]
 [  1.44300000e+02   2.95418540e-01   6.00000000e-03   2.90161332e-04]
 [  1.44300000e+02   2.35000000e-01   9.09838668e-03   9.09838668e-04]
 [  1.44300000e+02   1.74581460e-01   6.00000000e-03   2.90161332e-04]
 [  1.44300000e+02   2.35000000e-01   2.90161332e-03   9.09838668e-04]
 [  6.21152934e+01   2.35000000e-01   2.90161332e-03   2.90161332e-04]
 [  1.44300000e+02   2.95418540e-01   2.90161332e-03   6.00000000e-04]
 [  6.

Taking the output from the text file for dissipation, we have:

In [20]:
#Output = [15.9881,   16.5091 ,  16.0162   ,15.9950,   16.0310  , 16.4592  , 16.0958  , 15.8507 ,  16.0757  , 
#          15.9252  , 16.4301 ,  16.1259 ,  16.4682 ,  16.0501  , 16.2200]
Output_dissip = [24.8170119614, 23.7770471604, 24.6131673073, 24.9723698096, 24.6920894782, 23.7015914415, 24.6180488646, 
          25.2502586048, 24.576764997, 25.1656386185, 23.7951837649, 24.540205788, 23.7204920408, 24.514049663, 
         24.4823906956] 

Output_dissip = np.mat(Output_dissip)

[[ 24.81701196  23.77704716  24.61316731  24.97236981  24.69208948
   23.70159144  24.61804886  25.2502586   24.576765    25.16563862
   23.79518376  24.54020579  23.72049204  24.51404966  24.4823907 ]]


In [17]:
Output_dissip = Output_dissip.T
b = W * Output_dissip
A, normalizations = mat.rowNormalize(A)
b = np.dot(normalizations, b)

In [18]:
xn = mat.solveLeastSquares(A, b)
mean, variance = stats.compute_mean_variance(xn.T, IndexSet.getIndexSet(polybasis))
Sobol = stats.compute_first_order_Sobol_indices(xn.T, IndexSet.getIndexSet(polybasis))

In [19]:
print 'MEAN & VARIANCE IN DISSIPATION'
print mean, variance
print 'FIRST ORDER SOBOL SENSITIVITY INDICES FOR DISSIPATION'
print Sobol

MEAN & VARIANCE IN DISSIPATION
2.79342760885 0.0325064659306
FIRST ORDER SOBOL SENSITIVITY INDICES
[[ 0.0162397   0.31783944  0.3315129   0.33276632]]


In [25]:
Output_wl = [0.086849769577, 0.071449350565, 0.081941329874, 0.0897887852043, 0.0852774027735, 0.0701932846569, 
             0.0821369960904, 0.0950508601964, 0.081545121036, 0.0934180691838, 0.071064000949, 0.083252047188, 
             0.070388467051, 0.0827063908, 0.082069950178] 

Output_wl = np.mat(Output_wl)

In [26]:
Output_wl = Output_wl.T
b = W * Output_wl
A, normalizations = mat.rowNormalize(A)
b = np.dot(normalizations, b)

In [27]:
xn = mat.solveLeastSquares(A, b)
mean, variance = stats.compute_mean_variance(xn.T, IndexSet.getIndexSet(polybasis))
Sobol = stats.compute_first_order_Sobol_indices(xn.T, IndexSet.getIndexSet(polybasis))

In [28]:
print 'MEAN & VARIANCE IN WATER LEVEL'
print mean, variance
print 'FIRST ORDER SOBOL SENSITIVITY INDICES FOR WATER LEVEL'
print Sobol

MEAN & VARIANCE IN WATER LEVEL
0.00945510511158 1.20472293806e-06
FIRST ORDER SOBOL SENSITIVITY INDICES FOR WATER LEVEL
[[ 0.0095148   0.28265296  0.32349407  0.37760789]]


In [29]:
Output_ke=[9.28511113565, 8.59116120376, 9.20725286751, 9.22159875619, 9.19565064461, 8.5196991906, 9.15223514391, 
           9.38391902559, 9.15666537065, 9.31338865761, 8.57543965821, 9.0772799181, 8.53049860903, 9.17308676441, 
           8.98211942423]
Output_ke = np.mat(Output_ke)

In [30]:
Output_ke = Output_ke.T
b = W * Output_ke
A, normalizations = mat.rowNormalize(A)
b = np.dot(normalizations, b)

In [31]:
xn = mat.solveLeastSquares(A, b)
mean, variance = stats.compute_mean_variance(xn.T, IndexSet.getIndexSet(polybasis))
Sobol = stats.compute_first_order_Sobol_indices(xn.T, IndexSet.getIndexSet(polybasis))

In [32]:
print 'MEAN & VARIANCE IN KE CHANGE'
print mean, variance
print 'FIRST ORDER SOBOL SENSITIVITY INDICES FOR KE CHANGE'
print Sobol

MEAN & VARIANCE IN KE CHANGE
1.03448993893 0.00584536840325
FIRST ORDER SOBOL SENSITIVITY INDICES FOR KE CHANGE
[[ 0.01472562  0.29324053  0.31308905  0.36196057]]
